In [3]:
import pandas as pd
import json
import numpy as np
from collections import OrderedDict

In [166]:
# Specify the file to be read

filename = input("What is the file you'd like to read?")

## on my computer, I use the following
#filename = '/Users/willb/Documents/GitHub/efficient-learning-khan/code/eff data/efflearnkhan_18.csv'

fulldf = pd.read_csv(filename, sep='\t')

In [167]:
# generate the answer key

qdict = {}  # maps question => [answer, question ID, video number]

infile = open("../data analysis/astronomyquestions.tsv", "r")

parts = []
for line in infile.read().split("\n"):
    parts.append(line.split("\t"))

for line in parts:
    qdict[line[2]] = [line[3], line[0], line[1]]

In [168]:
# helper method

# updates the entry list given participant #num's questions (qsets) and answers (asets)
def grade(entries, num, qsets, asets):
    for set in range(3):    # for each of the 3 sets
        for q in range(13):    # for each of the 13 questions in the set
            question = qsets[set][q]
            entry = {"participantID":num}   # create an entry for the dataframe
            if question in qdict:    # check if we have the question...
                entry["qID"] = qdict[question][1]
                entry["set"] = set
                entry["video"] = qdict[question][2]
                if qdict[question][0] == asets[set][q]:
                    entry["correct?"] = 1
                else:
                    entry["correct?"] = 0
            else:     # ...and if not, output an error
                entry = {"participantID":num, "qID":-1, "set":set, "video":-1, "correct":-1}
            entries.append(entry)

In [169]:
# read and grade the file, and output a pandas dataframe called df

entries = []
    
for row in range(len(fulldf)):
    asets = []
    qsets = [] 
    for block in json.loads(fulldf.loc[row, 'datastring'])['data']: 
        # if the json block contains answers, put the text in subas
        if 'responses' in block['trialdata']:
            qsets.append([i.replace("&#8217;","\'") for i in OrderedDict(json.loads(block['trialdata']['responses'])).values()])
    
        # if the json block contains questions, put the text in subqs
        elif isinstance(block['trialdata'], type(list)):
            asets.append([qset['prompt'].replace("&#8217;","\'") for qset in block['trialdata']])
      
    if len(asets) == 3 and len(qsets) == 3:
        grade(entries, row + 1, asets, qsets)
    else:
        message = "Could not grade participant " + str(row) + ". Expected 3 sets of questions and 3 sets of answers,\n"
        message += "but found " + str(len(qsets)) +  " sets of questions and " + str(len(asets)) + " sets of answers." 
        print(message)
    
df = pd.DataFrame(entries)
df

Could not grade participant 1. Expected 3 sets of questions and 3 sets of answers,
but found 2 sets of questions and 3 sets of answers.


,correct?,participantID,qID,set,video
0,1,1,32,0,0
1,0,1,12,0,1
2,1,1,36,0,0
3,0,1,4,0,1
4,1,1,10,0,1
5,0,1,22,0,2
6,0,1,1,0,1
7,0,1,25,0,2
8,0,1,33,0,0
9,1,1,16,0,2
